In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name=&#39;AzureML&#39;, subscription_id=&#39;6560575d-fa06-4e7d-95fb-f962e74efd7a&#39;, resource_group=&#39;cody-eastus-rg&#39;)

In [2]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# training script
script_dir = prefix.joinpath("code", "lightgbm", "iris")
script_name = "train.py"

# environment file
environment_file = prefix.joinpath("environments", "lightgbm.txt")

# azure ml settings
environment_name = "hpo-lightgbm"
experiment_name = "hpo-lightgbm-optuna-loss"
compute_target = "cpu-cluster"

In [3]:
from azureml.core import Experiment, ScriptRunConfig, Environment

def objective(trial):
    ds = ws.datasets["iris"]
    env = Environment.from_pip_requirements(environment_name, environment_file)

    arguments = [
        "--data-dir",
        ds.as_mount(),
        "--num-boost-round",
        trial.suggest_int("--num-boost-round", 1, 100),
        "--boosting",
        trial.suggest_categorical(
            "--boosting", ["gbdt", "rf", "dart", "goss"]
        ),
        "--num-iterations",
        trial.suggest_int("--num-iterations", 1, 100),
        "--num-leaves",
        trial.suggest_int("--num-leaves", 2, 64),
        "--num-threads",
        trial.suggest_categorical("--num-threads", [1, 2, 4]),
        "--learning-rate",
        trial.suggest_loguniform("--learning-rate", 10e-5, 0.1),
        "--seed",
        trial.suggest_categorical("--seed", [1, 3, 5, 7, 11, 13, 42]),
    ]

    src = ScriptRunConfig(
        source_directory=script_dir,
        script=script_name,
        arguments=arguments,
        environment=env,
        compute_target=compute_target,
    )

    run = Experiment(ws, experiment_name).submit(src)
    run.wait_for_completion()
    if run.get_status() == "Completed":
        metrics = run.get_metrics()
        training_time = metrics["training_time"]
        loss = metrics["loss"]
        acc = metrics["accuracy"]

        return training_time
    else:
        return None

In [4]:
%%time

import joblib
import optuna
import dask_optuna

from dask.distributed import Client

c = Client()
print(c)
print(c.dashboard_link)

storage = dask_optuna.DaskStorage()
study = optuna.create_study(
    direction="minimize", study_name="dkdc", storage=storage
)
with joblib.parallel_backend("dask"):
    study.optimize(objective, n_trials=100, n_jobs=-1)

&lt;Client: &#39;tcp://127.0.0.1:61885&#39; processes=4 threads=8, memory=34.36 GB&gt;
http://127.0.0.1:8787/status
[I 2020-10-03 21:45:42,049] A new study created in memory with name: dkdc


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    &quot;error&quot;: {
        &quot;code&quot;: &quot;UserError&quot;,
        &quot;message&quot;: &quot;AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.&quot;,
        &quot;details&quot;: []
    },
    &quot;correlation&quot;: {
        &quot;operation&quot;: null,
        &quot;request&quot;: &quot;30a5432b49eb8b4a&quot;
    },
    &quot;environment&quot;: &quot;eastus&quot;,
    &quot;location&quot;: &quot;eastus&quot;,
    &quot;time&quot;: &quot;2020-10-04T04:49:23.534999Z&quot;
}
	InnerException None
	ErrorResponse 
{
    &quot;error&quot;: {
        &quot;message&quot;: &quot;Activity Failed:\n{\n    \&quot;error\&quot;: {\n        \&quot;code\&quot;: \&quot;UserError\&quot;,\n        \&quot;message\&quot;: \&quot;AzureMLCompute job failed.\\nJobFailed: Submitted script failed with a non-zero exit code; see the driver l

In [ ]:
study.best_params

In [ ]:
study.best_trial

In [ ]:
study.best_value